# Predicting 'GPS Speed' of Truck 2

#### Imports and global variables are defined here.

In [1]:
# Imports required for this notebook.
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import seaborn as sns

# Local path to the CSV file containing the data for truck one (1).
truckOnePath = "../data/trucks/truck1.csv"

# Local path to the CSV file containing the data for truck two (2).
truckTwoPath = "../data/trucks/truck2.csv"

#### Functions are defined here.

In [2]:
"""Uses Pandas's read_csv method to read a CSV file and returns a DataFrame of it to the notebook.
This function reads in all rows and defines the header row at index 0 by default."""
def readCsv(truck, records = None, headerIdx = 0):
    return pd.read_csv(truck, nrows = records, header = [headerIdx])

"""Takes a data frame of a truck (with inaccurate data) and the mean difference of a 
different truck (with accurate data) in order to attempt to predict or correct the GPS Speed value."""
def correctGps(truckDf, meanDiff):
    # TODO: Loop through Truck 2 Wheel-Based Vehicle Speed and create a predicted GPS Speed column based on the given mean.

#### Predicting GPS Speed for Truck 2

Based on our previous statistics, it is clear that the GPS Speed for Truck 2 is incorrect. Perhaps the component was broken, or maybe it was set to a different unit. Either way, it is not consistent with the Wheel-Based Vehicle Speed (which appears to be working correctly) and it is not consistent with the trends found in Truck 1, where both components were measuring relatively similar speeds at any given time.

In an attempt to rectify this, we will be using data from Truck 1 and analytically predict the GPS Speed values for Truck 2 given its Wheel-Based Vehicle Speed values.

In [3]:
# Read in the data for Truck 1.
truckOneDf = readCsv(truckOnePath)
truckOneDf = truckOneDf.ffill().bfill()

# Read in the data for Truck 2.
truckTwoDf = readCsv(truckTwoPath)
truckTwoDf = truckTwoDf.ffill().bfill()

To decide how to correct Truck 2's GPS Speed, we need to find the difference between the two components for Truck 1.

In [20]:
# Subtract the values of Wheel-Based Vehicle Speed from GPS Speed for Truck 1.
truckOneDiff = truckOneDf['GPS speed'].sub(truckOneDf['WheelBasedVehicleSpeed'])

# Calculate the average difference between all of the points of difference.
truckOneDiffMean = truckOneDiff.mean()

# Print the average difference of both components for Truck 1.
print(truckOneDiffMean)

-0.28539807628549124


Notice that the value is a negative number. This indicates that the GPS Speed component is, on average, reading a slightly higher value than the Wheel-Based Vehicle Speed component. Therefore, to begin predicting, or "correcting", the GPS Speed for Truck 2, we can start by simply adding that average to any given Wheel-Based Vehicle Speed value for Truck 2.

In [ ]:
correctedTruckTwoDf = correctGps(truckTwoDf, truckOneDiffMean)
correctedTruckTwoDf.head()